In [1]:
%cd ../..

/home/saplab/thaiminhpv/EFISS/AI-model-query-service


In [3]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
# mongoexport --uri mongodb+srv://efiss:*@efiss.wj88voa.mongodb.net/efiss --collection products --type json --out output-shopee.json --query '{"url": {"$regex": "https://shopee.vn"}}'

db_file = 'data/shopee/output-shopee.json'

data = []
with open(db_file, 'r') as f:
    for line in f:
        data.append(json.loads(line))

In [5]:
bdf = pd.DataFrame(data)
bdf.dropna(subset=['originalImages'], inplace=True)
df = bdf
# df = bdf.iloc[:1000]
df

,_id,title,url,price,description,images,group,__v,categories,originalImages
0,{'$oid': '6460ab0252e365505c04bdd6'},28801 ÁO THUN LEN CỘC TAY KHOÉT VAI,https://shopee.vn/product/134147548/23139530788,240000,cho len basic must have item nàng nào cũng nê...,[https://storage.googleapis.com/efiss/data/out...,shopee-thedelia,0,"[Thời Trang Nữ, Áo, Khác]",[https://down-vn.img.susercontent.com/file/vn-...
1,{'$oid': '6460aaee52e365505c04bd06'},Váy lụa tay bồng cutout lưng,https://shopee.vn/product/238591132/23240639596,480000,22.De chào bạn! 🤗\nĐể biết thêm thông tin chi ...,[https://storage.googleapis.com/efiss/data/out...,shopee-22decembrestore,0,"[Thời Trang Nữ, Đầm]",[https://down-vn.img.susercontent.com/file/vn-...
2,{'$oid': '6460aafe52e365505c04bda5'},Set váy yếm kèm áo tay bồng,https://shopee.vn/product/238591132/23136622940,580000,22.De chào bạn! 🤗\nĐể biết thêm thông tin chi ...,[https://storage.googleapis.com/efiss/data/out...,shopee-22decembrestore,0,"[Thời Trang Nữ, Đồ liền thân, Quần yếm]",[https://down-vn.img.susercontent.com/file/vn-...
3,{'$oid': '6460aaf952e365505c04bd6f'},Chân váy thô kẻ ô,https://shopee.vn/product/238591132/20877812095,294000,22.De chào bạn! 🤗\nĐể biết thêm thông tin chi ...,[https://storage.googleapis.com/efiss/data/out...,shopee-22decembrestore,0,"[Thời Trang Nữ, Váy]",[https://down-vn.img.susercontent.com/file/vn-...
4,{'$oid': '6460ab0552e365505c04be03'},891 ÁO THUN TRƠN NHIỀU MÀU,https://shopee.vn/product/134147548/23621140991,185000,Must have item nàng nào cũng nên có trong tủ đ...,[https://storage.googleapis.com/efiss/data/out...,shopee-thedelia,0,"[Thời Trang Nữ, Áo, Áo thun]",[https://down-vn.img.susercontent.com/file/vn-...
...,...,...,...,...,...,...,...,...,...,...
110576,{'$oid': '6462d711614eb2054537390d'},Áo sơ mi dài tay cổ sen 4LOVA cho bé gái kiểu ...,https://shopee.vn/product/244512865/5851475323,75000,- 4LOVA là shop uy tín chuyên cung cấp những s...,[https://storage.googleapis.com/efiss/data/out...,shopee-4lova,0,"[Thời trang trẻ em & trẻ sơ sinh, Quần áo bé g...",[https://down-vn.img.susercontent.com/file/d8f...
110577,{'$oid': '6462d717614eb2054537394f'},Quần thun cotton dài bo gấu 4LOVA cho bé trai ...,https://shopee.vn/product/244512865/19167117158,70000,- 4LOVA là shop uy tín chuyên cung cấp những s...,[https://storage.googleapis.com/efiss/data/out...,shopee-4lova,0,"[Thời trang trẻ em & trẻ sơ sinh, Quần áo bé t...",[https://down-vn.img.susercontent.com/file/sg-...
110578,{'$oid': '6462d71b614eb20545373976'},Scrunchies dây buộc tóc 4LOVA hình hoa cúc nhí...,https://shopee.vn/product/244512865/4246255828,8000,- 4LOVA là shop uy tín chuyên cung cấp những s...,[https://storage.googleapis.com/efiss/data/out...,shopee-4lova,0,"[Phụ Kiện Thời Trang, Phụ kiện tóc, Đồ buộc tó...",[https://down-vn.img.susercontent.com/file/105...
110579,{'$oid': '6462d715614eb2054537393d'},Áo thun dài tay unisex 4LOVA nhiều họa tiết ng...,https://shopee.vn/product/244512865/5957319697,79000,- 4LOVA là shop uy tín chuyên cung cấp những s...,[https://storage.googleapis.com/efiss/data/out...,shopee-4lova,0,"[Thời trang trẻ em & trẻ sơ sinh, Quần áo trẻ ...",[https://down-vn.img.susercontent.com/file/c16...


In [6]:
df = df[['title', 'images', 'categories']]
df['categories'] = df['categories'].apply(lambda x: " > ".join(x))
df

/tmp/ipykernel_1973168/3602483259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categories'] = df['categories'].apply(lambda x: " > ".join(x))


,title,images,categories
0,28801 ÁO THUN LEN CỘC TAY KHOÉT VAI,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Khác
1,Váy lụa tay bồng cutout lưng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đầm
2,Set váy yếm kèm áo tay bồng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đồ liền thân > Quần yếm
3,Chân váy thô kẻ ô,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Váy
4,891 ÁO THUN TRƠN NHIỀU MÀU,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Áo thun
...,...,...,...
110576,Áo sơ mi dài tay cổ sen 4LOVA cho bé gái kiểu ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé g...
110577,Quần thun cotton dài bo gấu 4LOVA cho bé trai ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé t...
110578,Scrunchies dây buộc tóc 4LOVA hình hoa cúc nhí...,[https://storage.googleapis.com/efiss/data/out...,Phụ Kiện Thời Trang > Phụ kiện tóc > Đồ buộc t...
110579,Áo thun dài tay unisex 4LOVA nhiều họa tiết ng...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo trẻ ...


In [7]:
df['categories'].value_counts()

Thời Trang Nữ > Đầm                                                       10281
Thời Trang Nữ > Áo > Áo thun                                               4906
Thời Trang Nữ > Áo > Áo sơ mi                                              4847
Thời Trang Nam > Áo > Áo sơ mi                                             4522
Phụ Kiện Thời Trang > Kính mắt > Gọng kính                                 4304
                                                                          ...  
Nhà cửa & Đời sống > Sắp xếp nhà cửa > Kệ giày, hộp giày                      1
Thực phẩm và đồ uống > Nguyên liệu nấu ăn > Gia vị & Hương liệu > Giấm        1
Thực phẩm và đồ uống > Đồ làm bánh > Bột nở và muối nở                        1
Sắc Đẹp > Chăm sóc nam giới > Sữa tắm & chăm sóc cơ thể                       1
Mẹ & Bé > Đồ dùng du lịch cho bé > Xe đẩy                                     1
Name: categories, Length: 702, dtype: int64

In [24]:
temp = df['categories'].value_counts()
temp = temp[temp > 10]
temp

Thời Trang Nữ > Đầm                                            10281
Thời Trang Nữ > Áo > Áo thun                                    4906
Thời Trang Nữ > Áo > Áo sơ mi                                   4847
Thời Trang Nam > Áo > Áo sơ mi                                  4522
Phụ Kiện Thời Trang > Kính mắt > Gọng kính                      4304
                                                               ...  
Mẹ & Bé > Chăm sóc sức khỏe bé > Chăm sóc răng miệng cho bé       12
Túi Ví Nữ > Phụ kiện túi > Túi đa ngăn tiện ích                   12
Nhà cửa & Đời sống > Bộ đồ bàn ăn > Cốc, ly, tách uống nước       12
Nhà cửa & Đời sống > Trang trí nhà cửa > Thảm trải sàn            11
Thể Thao & Dã Ngoại > Giày Thể Thao > Giày Tennis                 11
Name: categories, Length: 333, dtype: int64

### We will use 333 classes

How many ``images = []`` is missing?

In [13]:
sum(df['images'].apply(lambda x: len(x)) == 0) / len(df)

0.0

In [17]:
sum(bdf['originalImages'].apply(lambda x: len(x)) == 0) / len(df)

0.0

### Generate dataset for training

In [9]:
training_set = "data/shopee/full_database_info_22-7_filtered_shopee.txt"
train_df = []
with open(training_set, 'r') as f:
    for line in f:
        train_df.append(line)
train_df = pd.DataFrame(train_df)
train_df

,0
0,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
1,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
2,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
3,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
4,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
...,...
755887,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
755888,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
755889,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...
755890,/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic...


In [22]:
bdf.shape

(110434, 10)

In [25]:
temp.index

Index(['Thời Trang Nữ > Đầm', 'Thời Trang Nữ > Áo > Áo thun',
       'Thời Trang Nữ > Áo > Áo sơ mi', 'Thời Trang Nam > Áo > Áo sơ mi',
       'Phụ Kiện Thời Trang > Kính mắt > Gọng kính',
       'Thời Trang Nam > Áo > Áo thun', 'Thời Trang Nữ > Váy',
       'Thời Trang Nam > Áo > Áo polo',
       'Phụ Kiện Thời Trang > Kính mắt > Kính mát',
       'Thời Trang Nữ > Đồ lót > Quần lót',
       ...
       'Thể Thao & Dã Ngoại > Thời Trang Thể Thao & Dã Ngoại > Đồ Bơi > Đồ Bơi Khác',
       'Sắc Đẹp > Chăm sóc da mặt > Sản phẩm trị mụn',
       'Thể Thao & Dã Ngoại > Giày Thể Thao > Giày Tập Luyện',
       'Sắc Đẹp > Chăm sóc tóc > Sản phẩm dưỡng tóc',
       'Thực phẩm và đồ uống > Sữa - trứng > Sữa > Sữa tươi',
       'Mẹ & Bé > Chăm sóc sức khỏe bé > Chăm sóc răng miệng cho bé',
       'Túi Ví Nữ > Phụ kiện túi > Túi đa ngăn tiện ích',
       'Nhà cửa & Đời sống > Bộ đồ bàn ăn > Cốc, ly, tách uống nước',
       'Nhà cửa & Đời sống > Trang trí nhà cửa > Thảm trải sàn',
       'Thể Thao &

In [31]:
adf = df[df['categories'].isin(temp.index)]
adf

,title,images,categories
0,28801 ÁO THUN LEN CỘC TAY KHOÉT VAI,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Khác
1,Váy lụa tay bồng cutout lưng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đầm
2,Set váy yếm kèm áo tay bồng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đồ liền thân > Quần yếm
3,Chân váy thô kẻ ô,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Váy
4,891 ÁO THUN TRƠN NHIỀU MÀU,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Áo thun
...,...,...,...
110576,Áo sơ mi dài tay cổ sen 4LOVA cho bé gái kiểu ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé g...
110577,Quần thun cotton dài bo gấu 4LOVA cho bé trai ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé t...
110578,Scrunchies dây buộc tóc 4LOVA hình hoa cúc nhí...,[https://storage.googleapis.com/efiss/data/out...,Phụ Kiện Thời Trang > Phụ kiện tóc > Đồ buộc t...
110579,Áo thun dài tay unisex 4LOVA nhiều họa tiết ng...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo trẻ ...


In [32]:
# convert categories to class index
class_to_idx = {c: i for i, c in enumerate(temp.index)}
class_to_idx

{'Thời Trang Nữ > Đầm': 0,
 'Thời Trang Nữ > Áo > Áo thun': 1,
 'Thời Trang Nữ > Áo > Áo sơ mi': 2,
 'Thời Trang Nam > Áo > Áo sơ mi': 3,
 'Phụ Kiện Thời Trang > Kính mắt > Gọng kính': 4,
 'Thời Trang Nam > Áo > Áo thun': 5,
 'Thời Trang Nữ > Váy': 6,
 'Thời Trang Nam > Áo > Áo polo': 7,
 'Phụ Kiện Thời Trang > Kính mắt > Kính mát': 8,
 'Thời Trang Nữ > Đồ lót > Quần lót': 9,
 'Thời Trang Nữ > Đồ lót > Áo ngực': 10,
 'Thời trang trẻ em & trẻ sơ sinh > Quần áo trẻ em > Bộ quần áo': 11,
 'Thời trang trẻ em & trẻ sơ sinh > Quần áo bé gái > Váy': 12,
 'Thời Trang Nữ > Quần > Quần dài': 13,
 'Thời Trang Nam > Quần đùi': 14,
 'Thời Trang Nữ > Bộ > Đồ lẻ': 15,
 'Thời Trang Nữ > Áo > Áo hai dây và ba lỗ': 16,
 'Thời Trang Nữ > Đồ ngủ > Pyjama': 17,
 'Thời Trang Nam > Áo khoác > Áo khoác': 18,
 'Phụ Kiện Thời Trang > Thắt lưng': 19,
 'Thời Trang Nam > Đồ lót > Quần lót': 20,
 'Thời Trang Nữ > Quần đùi > Quần đùi': 21,
 'Thời trang trẻ em & trẻ sơ sinh > Quần áo bé gái > Com lê & đồ bộ': 22,
 'T

In [33]:
# convert categories to class index
adf['class_idx'] = adf['categories'].apply(lambda x: class_to_idx[x])
adf

/tmp/ipykernel_1973168/357757967.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adf['class_idx'] = adf['categories'].apply(lambda x: class_to_idx[x])


,title,images,categories,class_idx
0,28801 ÁO THUN LEN CỘC TAY KHOÉT VAI,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Khác,24
1,Váy lụa tay bồng cutout lưng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đầm,0
2,Set váy yếm kèm áo tay bồng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đồ liền thân > Quần yếm,112
3,Chân váy thô kẻ ô,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Váy,6
4,891 ÁO THUN TRƠN NHIỀU MÀU,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Áo thun,1
...,...,...,...,...
110576,Áo sơ mi dài tay cổ sen 4LOVA cho bé gái kiểu ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé g...,121
110577,Quần thun cotton dài bo gấu 4LOVA cho bé trai ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé t...,64
110578,Scrunchies dây buộc tóc 4LOVA hình hoa cúc nhí...,[https://storage.googleapis.com/efiss/data/out...,Phụ Kiện Thời Trang > Phụ kiện tóc > Đồ buộc t...,188
110579,Áo thun dài tay unisex 4LOVA nhiều họa tiết ng...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo trẻ ...,35


In [34]:
adf

,title,images,categories,class_idx
0,28801 ÁO THUN LEN CỘC TAY KHOÉT VAI,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Khác,24
1,Váy lụa tay bồng cutout lưng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đầm,0
2,Set váy yếm kèm áo tay bồng,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Đồ liền thân > Quần yếm,112
3,Chân váy thô kẻ ô,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Váy,6
4,891 ÁO THUN TRƠN NHIỀU MÀU,[https://storage.googleapis.com/efiss/data/out...,Thời Trang Nữ > Áo > Áo thun,1
...,...,...,...,...
110576,Áo sơ mi dài tay cổ sen 4LOVA cho bé gái kiểu ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé g...,121
110577,Quần thun cotton dài bo gấu 4LOVA cho bé trai ...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo bé t...,64
110578,Scrunchies dây buộc tóc 4LOVA hình hoa cúc nhí...,[https://storage.googleapis.com/efiss/data/out...,Phụ Kiện Thời Trang > Phụ kiện tóc > Đồ buộc t...,188
110579,Áo thun dài tay unisex 4LOVA nhiều họa tiết ng...,[https://storage.googleapis.com/efiss/data/out...,Thời trang trẻ em & trẻ sơ sinh > Quần áo trẻ ...,35


In [36]:
pdf = adf[['images', 'class_idx']]
pdf = pdf.explode('images')
pdf

,images,class_idx
0,https://storage.googleapis.com/efiss/data/outp...,24
0,https://storage.googleapis.com/efiss/data/outp...,24
0,https://storage.googleapis.com/efiss/data/outp...,24
0,https://storage.googleapis.com/efiss/data/outp...,24
1,https://storage.googleapis.com/efiss/data/outp...,0
...,...,...
110580,https://storage.googleapis.com/efiss/data/outp...,31
110580,https://storage.googleapis.com/efiss/data/outp...,31
110580,https://storage.googleapis.com/efiss/data/outp...,31
110580,https://storage.googleapis.com/efiss/data/outp...,31


In [56]:
pdf['images'] = "data/shopee/resize_600x600" + pdf['images'].str[48:]
pdf['images']

0         data/shopee/resize_600x600/shopee-thedelia/646...
0         data/shopee/resize_600x600/shopee-thedelia/646...
0         data/shopee/resize_600x600/shopee-thedelia/646...
0         data/shopee/resize_600x600/shopee-thedelia/646...
1         data/shopee/resize_600x600/shopee-22decembrest...
                                ...                        
110580    data/shopee/resize_600x600/shopee-4lova/6462d7...
110580    data/shopee/resize_600x600/shopee-4lova/6462d7...
110580    data/shopee/resize_600x600/shopee-4lova/6462d7...
110580    data/shopee/resize_600x600/shopee-4lova/6462d7...
110580    data/shopee/resize_600x600/shopee-4lova/6462d7...
Name: images, Length: 749059, dtype: object

In [59]:
pdf.set_index('images', inplace=True)

In [60]:
pdf

,class_idx
images,
data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_0_shopee_thedelia.jpeg,24
data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_1_shopee_thedelia.jpeg,24
data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_2_shopee_thedelia.jpeg,24
data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_3_shopee_thedelia.jpeg,24
data/shopee/resize_600x600/shopee-22decembrestore/6460aaee52e365505c04bd06_0_shopee_22decembrestore.jpeg,0
...,...
data/shopee/resize_600x600/shopee-4lova/6462d71a614eb2054537396d_3_shopee_4lova.jpeg,31
data/shopee/resize_600x600/shopee-4lova/6462d71a614eb2054537396d_4_shopee_4lova.jpeg,31
data/shopee/resize_600x600/shopee-4lova/6462d71a614eb2054537396d_5_shopee_4lova.jpeg,31


In [48]:
img_paths = "data/shopee/resize_600x600" + train_df[0].str[74:]
img_paths

0         data/shopee/resize_600x600/shopee-137kidsluxur...
1         data/shopee/resize_600x600/shopee-137kidsluxur...
2         data/shopee/resize_600x600/shopee-137kidsluxur...
3         data/shopee/resize_600x600/shopee-137kidsluxur...
4         data/shopee/resize_600x600/shopee-137kidsluxur...
                                ...                        
755887    data/shopee/resize_600x600/shopee-zunezx/6462a...
755888    data/shopee/resize_600x600/shopee-zunezx/6462a...
755889    data/shopee/resize_600x600/shopee-zunezx/6462a...
755890    data/shopee/resize_600x600/shopee-zunezx/6462a...
755891    data/shopee/resize_600x600/shopee-zunezx/6462a...
Name: 0, Length: 755892, dtype: object

In [63]:
pdf.loc['data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_1_shopee_thedelia.jpeg'].class_idx

24

In [73]:
mapping = pdf.to_dict()['class_idx']

In [74]:
mapping

{'data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_0_shopee_thedelia.jpeg': 24,
 'data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_1_shopee_thedelia.jpeg': 24,
 'data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_2_shopee_thedelia.jpeg': 24,
 'data/shopee/resize_600x600/shopee-thedelia/6460ab0252e365505c04bdd6_3_shopee_thedelia.jpeg': 24,
 'data/shopee/resize_600x600/shopee-22decembrestore/6460aaee52e365505c04bd06_0_shopee_22decembrestore.jpeg': 0,
 'data/shopee/resize_600x600/shopee-22decembrestore/6460aaee52e365505c04bd06_1_shopee_22decembrestore.jpeg': 0,
 'data/shopee/resize_600x600/shopee-22decembrestore/6460aaee52e365505c04bd06_2_shopee_22decembrestore.jpeg': 0,
 'data/shopee/resize_600x600/shopee-22decembrestore/6460aafe52e365505c04bda5_0_shopee_22decembrestore.jpeg': 112,
 'data/shopee/resize_600x600/shopee-22decembrestore/6460aafe52e365505c04bda5_1_shopee_22decembrestore.jpeg': 112,
 'data/shopee/resize_600x600/shopee-22de

In [80]:
train_df['class_idx']

Series([], Name: class_idx, dtype: int64)

In [99]:
def map_class_idx(img_path: str):
    img_path = img_path.strip()
    result = mapping.get(img_path, -1)
    return result

train_df['img_paths'] = img_paths.str.strip()
train_df['class_idx'] = train_df['img_paths'].progress_apply(map_class_idx)
train_df

  0%|          | 0/746341 [00:00<?, ?it/s]

,img_paths,class_idx
0,data/shopee/resize_600x600/shopee-137kidsluxur...,45
1,data/shopee/resize_600x600/shopee-137kidsluxur...,45
2,data/shopee/resize_600x600/shopee-137kidsluxur...,45
3,data/shopee/resize_600x600/shopee-137kidsluxur...,45
4,data/shopee/resize_600x600/shopee-137kidsluxur...,45
...,...,...
755887,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755888,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755889,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755890,data/shopee/resize_600x600/shopee-zunezx/6462a...,206


In [101]:
train_df = train_df[train_df['class_idx'] != -1]
train_df = train_df.drop(columns=[0])
train_df = train_df[['img_paths', 'class_idx']]
train_df

,img_paths,class_idx
0,data/shopee/resize_600x600/shopee-137kidsluxur...,45
1,data/shopee/resize_600x600/shopee-137kidsluxur...,45
2,data/shopee/resize_600x600/shopee-137kidsluxur...,45
3,data/shopee/resize_600x600/shopee-137kidsluxur...,45
4,data/shopee/resize_600x600/shopee-137kidsluxur...,45
...,...,...
755887,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755888,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755889,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755890,data/shopee/resize_600x600/shopee-zunezx/6462a...,206


In [103]:
train_df

,img_paths,class_idx
0,data/shopee/resize_600x600/shopee-137kidsluxur...,45
1,data/shopee/resize_600x600/shopee-137kidsluxur...,45
2,data/shopee/resize_600x600/shopee-137kidsluxur...,45
3,data/shopee/resize_600x600/shopee-137kidsluxur...,45
4,data/shopee/resize_600x600/shopee-137kidsluxur...,45
...,...,...
755887,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755888,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755889,data/shopee/resize_600x600/shopee-zunezx/6462a...,206
755890,data/shopee/resize_600x600/shopee-zunezx/6462a...,206


In [104]:
train_df.to_csv('data/shopee/database.txt', sep=' ', header=False, index=False)
test_df = train_df.sample(1000)
test_df.to_csv('data/shopee/test.txt', sep=' ', header=False, index=False)

In [105]:
!cat data/shopee/database.txt | head

data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373485_0_shopee_137kidsluxuryforkids.jpeg 45
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373485_1_shopee_137kidsluxuryforkids.jpeg 45
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373485_2_shopee_137kidsluxuryforkids.jpeg 45
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373485_3_shopee_137kidsluxuryforkids.jpeg 45
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373485_4_shopee_137kidsluxuryforkids.jpeg 45
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373485_5_shopee_137kidsluxuryforkids.jpeg 45
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373488_0_shopee_137kidsluxuryforkids.jpeg 97
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6462d673614eb20545373488_1_shopee_137kidsluxuryforkids.jpeg 97
data/shopee/resize_600x600/shopee-137kidsluxuryforkids/6